In [1]:
import pandas as pd
import numpy as np
import urllib3
import certifi
import json
import math
import tqdm

In [2]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [3]:
translate_key = 'trnsl.1.1.20181204T151552Z.9fcb0a1d8844bf3c.5507321eddbdb0415d2f60ed5318e9a642324aaf'
maps_key = '92488eba-dae2-41c9-8386-0d68e3591866'

In [4]:
train.address.count() - train.address_rus.count()

304

Заметка Попробовать if r.status == 200: заменить на try except

In [5]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

def geo(X, maps_key):
    problem_adresses = []
    address_rus = X.address_rus.values
    long = X.long.values
    lat = X.lat.values
    postal_code = np.zeros(X.shape[0], dtype=int)
    for i in tqdm.tqdm_notebook(range(X.shape[0])):
        r = http.request('GET', 'https://geocode-maps.yandex.ru/1.x/?apikey=' + maps_key + '&format=json&geocode=' + X.address[i])
        if r.status == 200:
            if math.isnan(X.long[i]):
                try:
                    coordinates = json.loads(r.data.decode('utf-8'))['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()
                    long[i] = float(coordinates[0])
                    lat[i] = float(coordinates[1])
                    street = json.loads(r.data.decode('utf-8'))['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['name']
                    region = json.loads(r.data.decode('utf-8'))['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['description']
                    address_rus[i] = street + ', ' + region
                except IndexError:
                    problem_adresses.append(i)
            try:
                postal_code[i] = int(json.loads(r.data.decode('utf-8'))['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['Address']['postal_code'])
            except BaseException: 
                postal_code[i] = 0
        else:
            if math.isnan(X.long[i]):
                problem_adresses.append(i)
    X.address_rus = address_rus
    X.long = long
    X.lat = lat
    X['postal_code'] = postal_code
    return X, problem_adresses

In [8]:
train, problem_adresses = geo(train, maps_key)

In [6]:
test, problem_test_adresses = geo(test, maps_key)

In [9]:
print(train.lat.count() - train.lat.count(), test.lat.count() - test.lat.count())

0 0
